In [ ]:
from IPython.display import Image, display
import os

# Display the logo image from the Assets directory
image_path = os.path.join("Assets", "HeatReuseEconomicsTool_Horizontal.jpg")

# Display the image with appropriate sizing
display(Image(filename=image_path, width=600))

# Interactive Analysis Tool

## User Guide

This notebook provides an easy-to-use interface for running calculations based on temperature and flow inputs.

### Instructions:
1. Run the setup cell below by clicking the ▶️ button or pressing Shift+Enter
2. Enter your values for temperatures (T1-T4) and flows (F1-F2) in the input fields
3. Click the "Run Calculation" button to process the data
4. View the results and visualizations below the button

**Note:** If you encounter any errors, please ensure all input values are numbers.

In [318]:
# Import required libraries
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import io
import base64
import math
from datetime import datetime
from pathlib import Path
import os

# Set up plotting configuration
%matplotlib inline
plt.style.use('ggplot')



In [319]:
# universal_float_convert

import re
import pandas as pd
import math

def universal_float_convert(value):
    """
    CORRECTED European-priority universal number parser
    Handles European CSV data formats correctly for ALLHX processing
    """
    
    # Handle None, NaN, and empty values
    if value is None:
        return 0.0
    
    try:
        if pd.isna(value):
            return 0.0
    except (TypeError, ValueError):
        pass
    
    # Handle numeric types that are already numbers
    if isinstance(value, (int, float)):
        if math.isnan(value) or math.isinf(value):
            return 0.0
        return float(value)
    
    # Convert to string and clean
    try:
        str_val = str(value).strip()
    except (UnicodeError, AttributeError):
        return 0.0
    
    # Handle empty string
    if not str_val:
        return 0.0
    
    # Handle special text cases
    special_cases = {
        'nan', 'none', 'null', 'n/a', 'na', '#n/a', '#value!', '#ref!', 
        '#div/0!', '#num!', 'inf', '-inf', 'infinity', '-infinity',
        'true', 'false', 'yes', 'no', 'error', 'err',
        'nichts', 'nul', 'erreur', 'infinito', 'niets', 'ingen'
    }
    if str_val.lower() in special_cases:
        return 0.0
    
    # Store original
    original_str = str_val
    
    # Handle scientific notation early
    scientific_patterns = [
        r'^-?[\d,]+\.?\d*[eE][+-]?\d+$',  # Standard: 1.5e3, 1,5e3
    ]
    for pattern in scientific_patterns:
        if re.match(pattern, str_val):
            try:
                sci_val = str_val.replace(',', '.')
                return float(sci_val)
            except (ValueError, OverflowError):
                continue
    
    # Handle percentage
    is_percentage = False
    if '%' in str_val:
        is_percentage = True
        str_val = str_val.replace('%', '')
    
    # Remove currency symbols
    currency_pattern = r'[$€£¥₹₽¢₦₪₨₩₫₡₵₸₴₺₼CHF|USD|EUR|GBP]'
    str_val = re.sub(currency_pattern, '', str_val, flags=re.IGNORECASE)
    
    # Remove text words
    text_patterns = [
        r'\b(ca\.?|etwa|circa|environ|ongeveer)\b',
        r'[()[\]{}"\']',
    ]
    for pattern in text_patterns:
        str_val = re.sub(pattern, '', str_val, flags=re.IGNORECASE)
    
    # Clean whitespace
    str_val = ' '.join(str_val.split())
    
    # Keep only numbers, separators, and signs
    str_val = re.sub(r'[^\d.,\s+\-\']', '', str_val).strip()
    
    if not str_val:
        return 0.0
    
    # Handle sign
    is_negative = False
    if str_val.startswith('-'):
        is_negative = True
        str_val = str_val[1:].strip()
    elif str_val.startswith('+'):
        str_val = str_val[1:].strip()
    
    if not str_val:
        return 0.0
    
    try:
        result = None
        
        # STRATEGY 1: Simple cases (no ambiguity)
        if re.match(r'^\d+$', str_val):
            # Pure integer: 123
            result = float(str_val)
            
        # STRATEGY 2: Clear multi-separator patterns (unambiguous)
        elif re.match(r'^\d{1,3}(\.\d{3})+,\d+$', str_val):
            # German: 1.234.567,89 (dots=thousands, comma=decimal)
            result = float(str_val.replace('.', '').replace(',', '.'))
            
        elif re.match(r'^\d{1,3}(\s\d{3})+,\d+$', str_val):
            # French: 1 234 567,89 (spaces=thousands, comma=decimal)
            result = float(str_val.replace(' ', '').replace(',', '.'))
            
        elif re.match(r'^\d{1,3}(\'\d{3})+\.\d+$', str_val):
            # Swiss: 1'234'567.89 (apostrophes=thousands, dot=decimal)
            result = float(str_val.replace('\'', ''))
            
        elif re.match(r'^\d{1,3}(,\d{3})+\.\d+$', str_val):
            # American: 1,234,567.89 (commas=thousands, dot=decimal)
            result = float(str_val.replace(',', ''))
            
        # STRATEGY 3: Thousands-only patterns (no decimal part)
        elif re.match(r'^\d{1,3}(\.\d{3})+$', str_val):
            # German thousands: 1.234.567
            result = float(str_val.replace('.', ''))
            
        elif re.match(r'^\d{1,3}(\s\d{3})+$', str_val):
            # French thousands: 1 234 567
            result = float(str_val.replace(' ', ''))
            
        elif re.match(r'^\d{1,3}(\'\d{3})+$', str_val):
            # Swiss thousands: 1'234'567
            result = float(str_val.replace('\'', ''))
            
        elif re.match(r'^\d{1,3}(,\d{3})+$', str_val):
            # American thousands: 1,234,567
            result = float(str_val.replace(',', ''))
            
        # STRATEGY 4: Single separator - CONTEXT-BASED FOR CSV DATA
        elif re.match(r'^\d+[.,]\d+$', str_val):
            # Single separator - need to determine if thousands or decimal
            if ',' in str_val:
                parts = str_val.split(',')
                integer_part = parts[0]
                fractional_part = parts[1]
                
                # FOR CSV DATA: Prioritize thousands interpretation for 3-digit patterns
                if len(fractional_part) == 3 and len(integer_part) >= 1:
                    # Likely thousands: 1,493 -> 1493, 11,024 -> 11024
                    result = float(integer_part + fractional_part)
                elif len(fractional_part) <= 2:
                    # Likely decimal: 1,5 -> 1.5, 123,45 -> 123.45
                    result = float(integer_part + '.' + fractional_part)
                else:
                    # 4+ digits after comma: definitely decimal
                    result = float(integer_part + '.' + fractional_part)
                    
            else:  # '.' in str_val
                parts = str_val.split('.')
                integer_part = parts[0]
                fractional_part = parts[1]
                
                # FOR CSV DATA: Prioritize thousands interpretation for 3-digit patterns
                if len(fractional_part) == 3 and len(integer_part) >= 1:
                    # Likely thousands: 1.493 -> 1493, 11.024 -> 11024
                    result = float(integer_part + fractional_part)
                elif len(fractional_part) <= 2 and len(integer_part) <= 3:
                    # Small numbers, likely decimal: 12.34 -> 12.34
                    result = float(str_val)
                else:
                    # Default to decimal for unclear cases
                    result = float(str_val)
        
        # STRATEGY 5: Mixed separators (both . and ,)
        elif '.' in str_val and ',' in str_val:
            last_dot = str_val.rfind('.')
            last_comma = str_val.rfind(',')
            
            if last_comma > last_dot:
                # Comma is last = decimal separator (European)
                before = str_val[:last_comma].replace('.', '').replace(',', '').replace(' ', '').replace('\'', '')
                after = str_val[last_comma + 1:]
                result = float(f"{before}.{after}")
            else:
                # Dot is last = decimal separator (American)
                before = str_val[:last_dot].replace('.', '').replace(',', '').replace(' ', '').replace('\'', '')
                after = str_val[last_dot + 1:]
                result = float(f"{before}.{after}")
        
        # STRATEGY 6: Fallback
        else:
            # Try to extract just the digits
            digits_only = re.sub(r'[^\d]', '', str_val)
            if digits_only:
                result = float(digits_only)
            else:
                return 0.0
        
        # Apply transformations
        if is_negative and result is not None:
            result = -result
        
        if is_percentage and result is not None:
            result = result / 100.0
        
        # Validate result
        if result is None or math.isnan(result) or math.isinf(result):
            return 0.0
            
        return result
        
    except (ValueError, TypeError, OverflowError):
        # Final fallback
        try:
            digits_only = re.sub(r'[^\d]', '', original_str)
            if digits_only:
                fallback = float(digits_only)
                if is_negative:
                    fallback = -fallback
                if is_percentage:
                    fallback = fallback / 100.0
                return fallback
            else:
                return 0.0
        except:
            return 0.0

In [320]:
# AllHX_Data

def lookup_allhx_data(power, t1, temp_diff, approach):
    """
    ALLHX lookup using proper data filtering and type consistency
    
    Args:
        power: System power in MW
        t1: Inlet temperature in °C
        temp_diff: Temperature difference in °C  
        approach: Approach value
    
    Returns:
        System data dictionary or None if not found
    """
    global csv_data
    
    t2 = t1 + temp_diff
    
    if 'ALLHX' not in csv_data:
        print("Error: ALLHX.csv not loaded")
        return None
    
    df = csv_data['ALLHX'].copy()
    
    print(f"ALLHX lookup: Power={power}, T1={t1}, TempDiff={temp_diff}, T2={t2}, Approach={approach}")
    
    # Clean data - remove header rows
    df = df[df['wha'].astype(str).str.strip() != 'A']
    df = df[df['wha'].astype(str).str.strip() != 'wha']
    
    # Convert to consistent numeric types
    numeric_columns = ['wha', 'T1', 'itdt', 'T2', 'TCSapp', 'F1', 'F2', 'T3', 'T4', 
                       'FWSapp', 'costHX', 'areaHX', 'Hxweight', 'CO2_Footprint']
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: float(universal_float_convert(x)))
    
    # Remove invalid rows
    valid_df = df[(df['wha'] > 0) & (df['T1'] > 0) & (df['itdt'] > 0) & (df['TCSapp'] > 0)]
    
    print(f"Valid data rows: {len(valid_df)}")
    
    if len(valid_df) == 0:
        print("No valid data after conversion")
        return None
    
    # Show available combinations for debugging
    print(f"Available combinations:")
    print(f"  Power (wha): {sorted(valid_df['wha'].unique())}")
    print(f"  T1: {sorted(valid_df['T1'].unique())}")
    print(f"  TempDiff (itdt): {sorted(valid_df['itdt'].unique())}")
    print(f"  Approach (TCSapp): {sorted(valid_df['TCSapp'].unique())}")
    
    # Convert inputs to float for consistent comparison
    power = float(power)
    t1 = float(t1)
    temp_diff = float(temp_diff)
    t2 = float(t2)
    approach = float(approach)
    
    # Exact match filtering
    filtered_df = valid_df[
        (valid_df['wha'] == power) & 
        (valid_df['T1'] == t1) & 
        (valid_df['itdt'] == temp_diff) & 
        (valid_df['T2'] == t2) & 
        (valid_df['TCSapp'] == approach)
    ]
    
    print(f"Exact matches found: {len(filtered_df)}")
    
    if filtered_df.empty:
        print(f"No exact match found for specified parameters")
        return None
    
    # Get first matching row
    row = filtered_df.iloc[0]
    
    result = {
        'power': power,
        'T1': t1,
        'T2': t2,
        'T3': row['T3'],
        'T4': row['T4'],
        'F1': row['F1'],
        'F2': row['F2'],
        'approach_tcs': approach,
        'approach_fws': row['FWSapp'] if 'FWSapp' in row else 0.0,
        'hx_unit': str(row['Unit']) if 'Unit' in row else '',
        'hx_cost': row['costHX'] if 'costHX' in row else 0.0,
        'hx_area': row['areaHX'] if 'areaHX' in row else 0.0,
        'hx_weight': row['Hxweight'] if 'Hxweight' in row else 0.0,
        'co2_footprint': row['CO2_Footprint'] if 'CO2_Footprint' in row else 0.0
    }
    
    print(f"Match found: F1={result['F1']}, F2={result['F2']}, HX_Cost=€{result['hx_cost']}")
    
    return result


In [321]:
# Lookup Functions

# ALSO UPDATE THESE FUNCTIONS to use universal_float_convert consistently

def get_system_sizing(system_data):
    """
    FIXED VERSION: Calculate system sizing with proper numeric handling
    """
    if not system_data:
        return None
    
    # Get pipe sizes based on flow rates
    pipe_size_f1 = None
    pipe_size_f2 = None
    
    if 'PIPSZ' in csv_data:
        pipsz_df = csv_data['PIPSZ'].copy()
        # Convert columns to numeric using universal parser
        pipsz_df.iloc[:, 0] = pipsz_df.iloc[:, 0].apply(universal_float_convert)  # Flow column
        pipsz_df.iloc[:, 1] = pipsz_df.iloc[:, 1].apply(universal_float_convert)  # Pipe size column
        
        # Find pipe size for F1
        for idx, row in pipsz_df.iterrows():
            flow_val = row.iloc[0]
            if flow_val >= system_data['F1']:
                pipe_size_f1 = row.iloc[1]
                break
        
        # Find pipe size for F2
        for idx, row in pipsz_df.iterrows():
            flow_val = row.iloc[0]
            if flow_val >= system_data['F2']:
                pipe_size_f2 = row.iloc[1]
                break
    
    # Get room size based on power
    room_size = None
    if 'ROOM' in csv_data:
        room_df = csv_data['ROOM'].copy()
        # Convert columns to numeric
        room_df.iloc[:, 0] = room_df.iloc[:, 0].apply(universal_float_convert)  # Power column
        room_df.iloc[:, 1] = room_df.iloc[:, 1].apply(universal_float_convert)  # Room size column
        
        for idx, row in room_df.iterrows():
            power_val = row.iloc[0]
            if power_val >= system_data['power']:
                room_size = row.iloc[1]
                break
    
    sizing_data = {
        'pipe_size_f1': pipe_size_f1 or 100,  # Default fallback
        'pipe_size_f2': pipe_size_f2 or 100,
        'room_size': room_size or 12.5,
        'primary_pipe_size': max(pipe_size_f1 or 100, pipe_size_f2 or 100)
    }
    
    return sizing_data

def calculate_system_costs(system_data, sizing_data):
    """
    FIXED VERSION: Calculate all system costs with proper numeric handling
    """
    if not system_data or not sizing_data:
        return None
    
    primary_pipe_size = sizing_data['primary_pipe_size']
    room_size = sizing_data['room_size']
    
    # Calculate pipe costs
    pipe_cost_per_meter = 0
    if 'PIPCOST' in csv_data:
        pipcost_df = csv_data['PIPCOST'].copy()
        # Convert to numeric
        pipcost_df.iloc[:, 0] = pipcost_df.iloc[:, 0].apply(universal_float_convert)  # Pipe size
        pipcost_df.iloc[:, 1] = pipcost_df.iloc[:, 1].apply(universal_float_convert)  # Cost
        
        for idx, row in pipcost_df.iterrows():
            size_val = row.iloc[0]
            if size_val >= primary_pipe_size:
                pipe_cost_per_meter = row.iloc[1]
                break
    
    total_pipe_length = room_size * 3  # Based on meeting notes
    total_pipe_cost = pipe_cost_per_meter * total_pipe_length
    
    # Calculate valve costs
    control_valve_cost = 0
    isolation_valve_cost = 0
    
    if 'CVALV' in csv_data:
        cvalv_df = csv_data['CVALV'].copy()
        # Convert cost column to numeric
        cvalv_df.iloc[:, 1] = cvalv_df.iloc[:, 1].apply(universal_float_convert)
        
        # Look for exact match on pipe size
        pipe_size_str = str(int(primary_pipe_size))
        for idx, row in cvalv_df.iterrows():
            if str(row.iloc[0]).strip() == pipe_size_str:
                control_valve_cost = row.iloc[1]
                break
    
    if 'IVALV' in csv_data:
        ivalv_df = csv_data['IVALV'].copy()
        # Convert cost column to numeric
        ivalv_df.iloc[:, 1] = ivalv_df.iloc[:, 1].apply(universal_float_convert)
        
        # Look for exact match on pipe size
        pipe_size_str = str(int(primary_pipe_size))
        for idx, row in ivalv_df.iterrows():
            if str(row.iloc[0]).strip() == pipe_size_str:
                isolation_valve_cost = row.iloc[1]
                break
    
    total_valve_cost = (control_valve_cost + isolation_valve_cost) * 4  # 4 of each type
    
    # Other costs
    hx_cost = system_data['hx_cost']
    pump_cost = system_data['power'] * 5000  # Estimated
    installation_cost = 10000  # Placeholder
    total_cost = total_pipe_cost + total_valve_cost + hx_cost + pump_cost + installation_cost
    
    cost_data = {
        'pipe_cost_per_meter': pipe_cost_per_meter,
        'total_pipe_length': total_pipe_length,
        'total_pipe_cost': total_pipe_cost,
        'control_valve_cost': control_valve_cost,
        'isolation_valve_cost': isolation_valve_cost,
        'total_valve_cost': total_valve_cost,
        'hx_cost': hx_cost,
        'pump_cost': pump_cost,
        'installation_cost': installation_cost,
        'total_cost': total_cost
    }
    
    return cost_data

def get_complete_system_analysis(power, t1, temp_diff, approach):
    """
    complete system analysis using corrected functions
    """
    print(f"\n🔧 COMPLETE SYSTEM ANALYSIS")
    print(f"Input: {power}MW, {t1}°C, +{temp_diff}°C, approach {approach}")
    
    # Step 1: Get system data from ALLHX
    system_data = lookup_allhx_data(power, t1, temp_diff, approach)
    if not system_data:
        print("ALLHX lookup failed")
        return None
    
    print("✅ ALLHX lookup successful")
    
    # Step 2: Calculate sizing using floor lookup
    sizing_data = floor_system_sizing(system_data)
    if not sizing_data:
        print("Floor sizing failed")
        return None
    
    print("✅ Floor sizing successful")
    
    # Step 3: Calculate costs using floor lookup
    cost_data = floor_cost_calculation(system_data, sizing_data)
    if not cost_data:
        print("Floor cost calculation failed")
        return None
    
    print("✅ Floor cost calculation successful")
    
    # Combine all data
    complete_analysis = {
        'system': system_data,
        'sizing': sizing_data,
        'costs': cost_data,
        'summary': {
            'power_mw': system_data['power'],
            't1_celsius': system_data['T1'],
            't2_celsius': system_data['T2'],
            't3_celsius': system_data['T3'],
            't4_celsius': system_data['T4'],
            'f1_flow': system_data['F1'],
            'f2_flow': system_data['F2'],
            'pipe_size': sizing_data['primary_pipe_size'],
            'room_size': sizing_data['room_size'],
            'total_cost_eur': round(cost_data['total_cost'])
        }
    }
    
    return complete_analysis

    """
    PRODUCTION VERSION: Main function to get complete system analysis
    """
    # Step 1: Get basic system data from ALLHX
    system_data = lookup_allhx_data(power, t1, temp_diff, approach)
    if not system_data:
        return None
    
    # Step 2: Calculate sizing
    sizing_data = get_system_sizing(system_data)
    if not sizing_data:
        return None
    
    # Step 3: Calculate costs
    cost_data = calculate_system_costs(system_data, sizing_data)
    if not cost_data:
        return None
    
    # Combine all data
    complete_analysis = {
        'system': system_data,
        'sizing': sizing_data,
        'costs': cost_data,
        'summary': {
            'power_mw': system_data['power'],
            't1_celsius': system_data['T1'],
            't2_celsius': system_data['T2'],
            't3_celsius': system_data['T3'],
            't4_celsius': system_data['T4'],
            'f1_flow': system_data['F1'],
            'f2_flow': system_data['F2'],
            'pipe_size': sizing_data['primary_pipe_size'],
            'room_size': sizing_data['room_size'],
            'total_cost_eur': round(cost_data['total_cost'])
        }
    }
    
    return complete_analysis

def validate_user_inputs(power, t1, temp_diff, approach):
    """
    PRODUCTION VERSION: Validate user inputs
    """
    errors = []
    
    if power not in [1, 2, 3, 4, 5]:
        errors.append(f"Power must be 1, 2, 3, 4, or 5 MW (got {power})")
    
    if t1 not in [20, 30, 45]:
        errors.append(f"T1 must be 20, 30, or 45°C (got {t1})")
    
    if temp_diff not in [10, 12, 14]:
        errors.append(f"Temperature difference must be 10, 12, or 14°C (got {temp_diff})")
    
    if approach not in [2, 3, 5]:
        errors.append(f"Approach must be 2, 3, or 5 (got {approach})")
    
    return errors

In [322]:
# Formulas
def mw_divd(F1, T1, T2):
    mw_value = get_MW(F1, T1, T2)
    result = mw_value / 1000000
    rounded = round(result, 2)
    return rounded


def get_itdt(T1,T2):
    # Temperature Difference Across IT side of Heat Exchanger
    x=T2-T1
    # print(x)
    return T2-T1

def get_PipeSize_Suggested(F1):
    x=get_lookup_value("PIPSZ",F1)
    return x

def get_MW(F1, T1,T2):
    # F1 = l/m
    # l/m / 60 seconds * 4186 J/kg * itdt
    itdt = get_itdt(T1,T2)
    formula = F1 / 60 * 4186 * itdt
    return formula

def get_PipeCost_perMeter(F1):
    pss = get_PipeSize_Suggested(F1)
    pcpm = get_lookup_value("PIPCOST",pss,1)
    return pcpm
    
def get_PipeLength(F1,T1,T2):
    mw = mw_divd(F1, T1, T2)
    pipelength = get_lookup_value("ROOM",mw)    
    return pipelength

def get_PipeCost_Total(F1,T1,T2):
    ttl=get_PipeCost_perMeter(F1)*get_PipeLength(F1,T1,T2)
    return ttl
    

In [323]:
# Interface Setup and Widget Creation

from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Create the header
# display(HTML("""
# <div style="background-color: #f0f0f0; padding: 15px; border-radius: 8px; margin-bottom: 15px;">
#     <h3 style="margin-top: 0; color: #333;">🔧 System Configuration</h3>
#     <p style="margin-bottom: 0; color: #666;">Select system parameters from the dropdowns below, then click "Calculate System"</p>
# </div>
# """))

# Create dropdown widgets with the exact values from the specifications
style = {'description_width': '140px'}
layout = widgets.Layout(width='300px')

# A. Power/Capacity dropdown (1, 2, 3, 4, 5)
power_widget = widgets.Dropdown(
    options=[1, 2, 3, 4, 5],
    value=1,
    description='Power/Capacity:',
    tooltip='System power capacity in MW',
    style=style,
    layout=layout
)

# B. T1 Temperature dropdown (20, 30, 45)
t1_widget = widgets.Dropdown(
    options=[20, 30, 45],
    value=20,
    description='T1 Temperature:',
    tooltip='T1 outlet temperature in °C',
    style=style,
    layout=layout
)

# C. Temperature Rise dropdown (10, 12, 14)
temp_diff_widget = widgets.Dropdown(
    options=[10, 12, 14],
    value=10,
    description='Temperature Rise:',
    tooltip='Temperature difference (T2 - T1) in °C',
    style=style,
    layout=layout
)

# E. Approach dropdown (2, 3, 5)
approach_widget = widgets.Dropdown(
    options=[2, 3, 5],
    value=2,
    description='Approach:',
    tooltip='System approach value',
    style=style,
    layout=layout
)

# Create output areas for results
system_params_output = widgets.Output()
cost_analysis_output = widgets.Output()
charts_output = widgets.Output()

# Calculate System button
calculate_button = widgets.Button(
    description='Calculate System',
    button_style='success',
    tooltip='Calculate system parameters and costs',
    icon='calculator',
    layout=widgets.Layout(width='200px', height='45px', margin='10px 0')
)

# print("Widgets created successfully!")

In [324]:
# Dictionary to store all dataframes
csv_data = {}

# Function to get a specific dataframe
def get_dataframe(csv_name):
    """
    Get a specific dataframe by name.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    
    Returns:
    pandas.DataFrame: The dataframe, or None if not found
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    return csv_data[csv_name]

def load_csv_files(data_dir="Data"):
    """
    Load all CSV files from the specified directory.
    
    Parameters:
    data_dir (str): Path to the directory containing CSV files
    
    Returns:
    dict: Dictionary of dataframes with normalized names as keys
    """
    global csv_data
    
    try:
        # Get all CSV files in the directory
        csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
        
        # Load each CSV file into a dataframe
        for file in csv_files:
            # Create a normalized name for the dataframe (without .csv extension, uppercase)
            df_name = os.path.splitext(file)[0].upper()
            file_path = os.path.join(data_dir, file)
            
            try:
                # Try to read the CSV file
                csv_data[df_name] = pd.read_csv(file_path)
                print(f"Loaded: {file} as {df_name}")
            except Exception as e:
                # Try with different separators if automatic detection fails
                try:
                    csv_data[df_name] = pd.read_csv(file_path, sep=';')
                    print(f"Loaded: {file} as {df_name} (using semicolon separator)")
                except:
                    try:
                        csv_data[df_name] = pd.read_csv(file_path, sep='\t')
                        print(f"Loaded: {file} as {df_name} (using tab separator)")
                    except Exception as e2:
                        print(f"Failed to load {file}: {e2}")
        
        return csv_data
    
    except Exception as e:
        print(f"Error loading CSV files: {e}")
        return {}

def get_lookup_value(csv_name, lookup_value, col_index_lookup=0, col_index_return=1):
    """
    Look up a value in a CSV file based on finding the first value 
    in col_index_lookup that is >= lookup_value, then return the 
    corresponding value from col_index_return.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (will be compared against col_index_lookup)
    col_index_lookup (int): Index of column to search in (default: 0)
    col_index_return (int/str/list): Index of column(s) to return value from (default: 1)
                                    Can be an integer, column name, or list of integers/names
    
    Returns:
    The value from col_index_return corresponding to the first row where
    col_index_lookup >= lookup_value, or None if not found.
    If col_index_return is a list, returns a dictionary with column names/indices as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Get column name for lookup column
    lookup_col = df.columns[col_index_lookup] if isinstance(col_index_lookup, int) else col_index_lookup
    
    # Sort the dataframe by the lookup column to ensure proper comparison
    df_sorted = df.sort_values(by=lookup_col)
    
    # Find rows where lookup column >= lookup_value
    matching_rows = df_sorted[df_sorted[lookup_col] >= lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row (which will be the smallest value >= lookup_value)
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(col_index_return, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in col_index_return:
            col_name = df.columns[col] if isinstance(col, int) else col
            if col_name in df.columns:
                result[col_name] = matched_row[col_name]
            else:
                print(f"Warning: Column '{col_name}' not found in '{csv_name}'")
        return result
    else:
        # Return a single column
        return_col = df.columns[col_index_return] if isinstance(col_index_return, int) else col_index_return
        if return_col in df.columns:
            return matched_row[return_col]
        else:
            print(f"Column '{return_col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None

def get_lookup_value_by_name(csv_name, lookup_value, lookup_col_name, return_col_name):
    """
    Look up a value in a CSV file based on finding the first value 
    in lookup_col_name that is >= lookup_value, then return the 
    corresponding value from return_col_name.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (will be compared against lookup_col_name)
    lookup_col_name (str): Name of column to search in
    return_col_name (str/list): Name of column(s) to return value from.
                               Can be a single column name or list of names.
    
    Returns:
    The value from return_col_name corresponding to the first row where
    lookup_col_name >= lookup_value, or None if not found.
    If return_col_name is a list, returns a dictionary with column names as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Check if lookup column exists
    if lookup_col_name not in df.columns:
        print(f"Column '{lookup_col_name}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
        return None
    
    # If return_col_name is a list, check all columns exist
    if isinstance(return_col_name, (list, tuple)):
        for col in return_col_name:
            if col not in df.columns:
                print(f"Column '{col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
                # Continue anyway, will just skip this column
    else:
        # Single column - check it exists
        if return_col_name not in df.columns:
            print(f"Column '{return_col_name}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None
    
    # Sort the dataframe by the lookup column to ensure proper comparison
    df_sorted = df.sort_values(by=lookup_col_name)
    
    # Find rows where lookup column >= lookup_value
    matching_rows = df_sorted[df_sorted[lookup_col_name] >= lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row (which will be the smallest value >= lookup_value)
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(return_col_name, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in return_col_name:
            if col in df.columns:
                result[col] = matched_row[col]
        return result
    else:
        # Return a single column
        return matched_row[return_col_name]

def get_exact_match(csv_name, lookup_value, lookup_col_index=0, return_col_index=1):
    """
    Look up a value in a CSV file based on finding an exact match
    in lookup_col_index, then return the corresponding value from return_col_index.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (must match exactly)
    lookup_col_index (int/str): Index or name of column to search in (default: 0)
    return_col_index (int/str/list): Index or name of column(s) to return value from (default: 1)
                                    Can be an integer, column name, or list of integers/names
    
    Returns:
    The value from return_col_index corresponding to the row where
    lookup_col_index == lookup_value, or None if not found.
    If return_col_index is a list, returns a dictionary with column names/indices as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Get column name for lookup column
    lookup_col = df.columns[lookup_col_index] if isinstance(lookup_col_index, int) else lookup_col_index
    
    # Find rows where lookup column == lookup_value
    matching_rows = df[df[lookup_col] == lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(return_col_index, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in return_col_index:
            col_name = df.columns[col] if isinstance(col, int) else col
            if col_name in df.columns:
                result[col_name] = matched_row[col_name]
            else:
                print(f"Warning: Column '{col_name}' not found in '{csv_name}'")
        return result
    else:
        # Return a single column
        return_col = df.columns[return_col_index] if isinstance(return_col_index, int) else return_col_index
        if return_col in df.columns:
            return matched_row[return_col]
        else:
            print(f"Column '{return_col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None

def show_csv_info(csv_name=None):
    """
    Display information about loaded CSV files.
    
    Parameters:
    csv_name (str, optional): Name of specific CSV to display info for.
                             If None, displays info for all CSVs.
    """
    global csv_data
    
    if csv_name:
        # Normalize the CSV name
        csv_name = csv_name.upper()
        
        # Check if the CSV has been loaded
        if csv_name not in csv_data:
            print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
            return
        
        # Display info for the specified CSV
        df = csv_data[csv_name]
        print(f"\n=== CSV: {csv_name} ===")
        print(f"Shape: {df.shape}")
        print("\nColumns:")
        for i, col in enumerate(df.columns):
            print(f"  {i}: {col}")
        print("\nFirst 5 rows:")
        print(df.head())
    else:
        # Display info for all CSVs
        for name, df in csv_data.items():
            print(f"\n=== CSV: {name} ===")
            print(f"Shape: {df.shape}")
            print("\nColumns:")
            for i, col in enumerate(df.columns):
                print(f"  {i}: {col}")
            print("\nFirst 5 rows:")
            print(df.head())

# Example usage
# First load all CSVs
# data_dir = "Data"  # Update this to your data directory
# load_csv_files(data_dir)

# Example of how to use the lookup functions
# pipe_size = 2.0
# pipe_cost = get_lookup_value("PIPCOST", pipe_size)
# print(f"Cost for pipe size {pipe_size}: {pipe_cost}")

# Example of how to use exact match lookup
# valve_type = "GATE"
# valve_cost = get_exact_match("CVALV", valve_type)
# print(f"Cost for valve type {valve_type}: {valve_cost}")

# Example of calculations using values from CSVs
# def calculate_system_cost(pipe_size, pipe_length, valve_type=None):
#     # Get pipe cost per unit length
#     pipe_cost_per_unit = get_lookup_value("PIPCOST", pipe_size)
#     total_pipe_cost = pipe_cost_per_unit * pipe_length
#     
#     # Add valve cost if specified
#     total_valve_cost = 0
#     if valve_type:
#         valve_cost = get_exact_match("CVALV", valve_type)
#         if valve_cost:
#             total_valve_cost = valve_cost
#     
#     # Calculate total cost
#     total_cost = total_pipe_cost + total_valve_cost
#     return total_cost
# 
# # Calculate system cost
# system_cost = calculate_system_cost(2.0, 100, "GATE")
# print(f"Total system cost: ${system_cost:.2f}")

load_csv_files()
# show_csv_info()

Loaded: ALLHX.csv as ALLHX
Loaded: CVALV.csv as CVALV
Loaded: HX.csv as HX
Loaded: IVALV.csv as IVALV
Loaded: JOINTS.csv as JOINTS
Loaded: MW Price Data.csv as MW PRICE DATA
Loaded: PIPCOST.csv as PIPCOST
Loaded: PIPSZ.csv as PIPSZ
Loaded: ROOM.csv as ROOM


{'ALLHX':      wha  T1 itdt  T2 TCSapp     F1  T4  T3     F2 FWSapp  ... Unnamed: 15  \
 0    1.0  20   10  30      2  1,493  18  28  1,440      2  ...         NaN   
 1    1.0  20   10  30      3  1,493  17  27  1,440      3  ...         NaN   
 2    1.0  20   10  30      5  1,493  15  25  1,440      5  ...         NaN   
 3    1.0  20   12  32      2  1,244  18  30  1,200      2  ...         NaN   
 4    1.0  20   12  32      3  1,244  17  29  1,200      3  ...         NaN   
 ..   ...  ..  ...  ..    ...    ...  ..  ..    ...    ...  ...         ...   
 131  5.0  45   12  57      5  6,209  40  52  6,061      5  ...         NaN   
 132  5.0  45   14  59      2  5,322  43  57  5,195      2  ...         NaN   
 133  5.0  45   14  59      3  5,322  42  56  5,195      3  ...         NaN   
 134  5.0  45   14  59      5  5,322  40  54  5,195      5  ...         NaN   
 135    A   B   DT   C     D       Z   Z   Z      Z      Z  ...         NaN   
 
     Unnamed: 16 Unnamed: 17 Unnamed: 18 

In [325]:
# Chart Creation Function

def create_system_charts(analysis):
    """Create visualization charts for the system analysis"""
    try:
        fig, axs = plt.subplots(2, 2, figsize=(14, 10))
        
        system = analysis['system']
        costs = analysis['costs']
        
        # 1. Temperature flow chart
        temps = ['T1\n(Out to TCS)', 'T2\n(In from TCS)', 'T3\n(Out to Consumer)', 'T4\n(In from Consumer)']
        temp_values = [system['T1'], system['T2'], system['T3'], system['T4']]
        colors_temp = ['#ff9999', '#ff6666', '#66b3ff', '#3399ff']
        
        bars1 = axs[0, 0].bar(temps, temp_values, color=colors_temp)
        axs[0, 0].set_title('System Temperatures', fontsize=14, fontweight='bold')
        axs[0, 0].set_ylabel('Temperature (°C)')
        for i, v in enumerate(temp_values):
            axs[0, 0].text(i, v + max(temp_values)*0.02, f"{v}°C", ha='center', fontweight='bold')
        
        # 2. Flow rates comparison
        flows = ['F1\n(TCS Flow)', 'F2\n(FWS Flow)']
        flow_values = [system['F1'], system['F2']]
        colors_flow = ['#99ff99', '#66cc66']
        
        bars2 = axs[0, 1].bar(flows, flow_values, color=colors_flow)
        axs[0, 1].set_title('Flow Rates', fontsize=14, fontweight='bold')
        axs[0, 1].set_ylabel('Flow Rate (l/m)')
        for i, v in enumerate(flow_values):
            axs[0, 1].text(i, v + max(flow_values)*0.02, f"{v:,}", ha='center', fontweight='bold')
        
        # 3. Cost breakdown
        cost_items = ['Pipe\nCost', 'Heat\nExchanger', 'Valves', 'Pump', 'Installation']
        cost_values = [
            costs['total_pipe_cost'],
            costs['hx_cost'],
            costs['total_valve_cost'],
            costs['pump_cost'],
            costs['installation_cost']
        ]
        colors_cost = ['#ffcc99', '#ff9999', '#cc99ff', '#99ccff', '#ffff99']
        
        bars3 = axs[1, 0].bar(cost_items, cost_values, color=colors_cost)
        axs[1, 0].set_title('Cost Breakdown', fontsize=14, fontweight='bold')
        axs[1, 0].set_ylabel('Cost (€)')
        for i, v in enumerate(cost_values):
            axs[1, 0].text(i, v + max(cost_values)*0.02, f"€{v:,.0f}", ha='center', fontweight='bold', rotation=45)
        
        # 4. System summary metrics
        metrics = ['Power\n(MW)', 'Temp Rise\n(°C)', 'Pipe Size', 'Room Size\n(m)']
        metric_values = [
            system['power'],
            system['T2'] - system['T1'],
            analysis['sizing']['primary_pipe_size'],
            analysis['sizing']['room_size']
        ]
        colors_metrics = ['#ff6666', '#66ff66', '#6666ff', '#ffcc66']
        
        bars4 = axs[1, 1].bar(metrics, metric_values, color=colors_metrics)
        axs[1, 1].set_title('System Metrics', fontsize=14, fontweight='bold')
        axs[1, 1].set_ylabel('Values')
        for i, v in enumerate(metric_values):
            axs[1, 1].text(i, v + max(metric_values)*0.02, f"{v}", ha='center', fontweight='bold')
        
        plt.suptitle(f'Heat Reuse System Analysis - {system["power"]}MW System', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        display(HTML(f"""
        <div style="background-color: #fff3cd; color: #856404; padding: 10px; border-radius: 5px;">
            <strong>Chart Warning:</strong> Could not generate charts: {str(e)}
        </div>
        """))

# print("Chart function defined successfully!")


In [326]:
# Button Click Handler Function
def on_calculate_click(b):
    with system_params_output:
        clear_output()
    with cost_analysis_output:
        clear_output()
    with charts_output:
        clear_output()
        
    try:
        # Get values from dropdown widgets
        power = power_widget.value
        t1 = t1_widget.value
        temp_diff = temp_diff_widget.value
        approach = approach_widget.value
        
        # Validate inputs first
        errors = validate_user_inputs(power, t1, temp_diff, approach)
        if errors:
            with system_params_output:
                error_list = "<br>".join([f"• {error}" for error in errors])
                display(HTML(f"""
                <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                    <strong>Input Validation Errors:</strong><br>
                    {error_list}
                </div>
                """))
            return
        
        # Get complete system analysis using PRODUCTION functions
        analysis = get_complete_system_analysis(power, t1, temp_diff, approach)
        
        if not analysis:
            with system_params_output:
                display(HTML("""
                <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                    <strong>Error:</strong> No data found for the selected parameters. Please try a different combination.
                </div>
                """))
            return
        
        # Display system parameters
        with system_params_output:
            system = analysis['system']
            display(HTML(f"""
            <div style="background-color: white; padding: 15px; border-radius: 8px; border: 2px solid #4CAF50; margin: 10px 0;">
                <h3 style="color: #4CAF50; margin-top: 0;">📊 System Parameters (Auto-Calculated)</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">T1 (Outlet to TCS):</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{system['T1']}°C</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">T2 (Inlet from TCS):</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{system['T2']}°C</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">T3 (Outlet to Consumer):</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{system['T3']}°C</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">T4 (Inlet from Consumer):</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{system['T4']}°C</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">F1 (TCS Flow Rate):</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{system['F1']:,} l/m</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold;">F2 (FWS Flow Rate):</td>
                        <td style="padding: 8px;">{system['F2']:,} l/m</td></tr>
                </table>
            </div>
            """))
        
        # Display cost analysis
        with cost_analysis_output:
            costs = analysis['costs']
            sizing = analysis['sizing']
            display(HTML(f"""
            <div style="background-color: white; padding: 15px; border-radius: 8px; border: 2px solid #2196F3; margin: 10px 0;">
                <h3 style="color: #2196F3; margin-top: 0;">💰 Cost Analysis</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">Room Size:</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{sizing['room_size']} m</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">Suggested Pipe Size:</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">{sizing['primary_pipe_size']}</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">Pipe Cost per Meter:</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">€{costs['pipe_cost_per_meter']:.2f}/m</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">Total Pipe Cost:</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">€{costs['total_pipe_cost']:,.0f}</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">Heat Exchanger Cost:</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">€{costs['hx_cost']:,.0f}</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 1px solid #eee;">Valve Costs:</td>
                        <td style="padding: 8px; border-bottom: 1px solid #eee;">€{costs['total_valve_cost']:,.0f}</td></tr>
                    <tr><td style="padding: 8px; font-weight: bold; border-bottom: 2px solid #333;">Pump Cost:</td>
                        <td style="padding: 8px; border-bottom: 2px solid #333;">€{costs['pump_cost']:,.0f}</td></tr>
                    <tr><td style="padding: 10px; font-weight: bold; font-size: 18px; color: #f44336;">TOTAL SYSTEM COST:</td>
                        <td style="padding: 10px; font-weight: bold; font-size: 18px; color: #f44336;">€{costs['total_cost']:,.0f}</td></tr>
                </table>
            </div>
            """))
        
        # Create and display charts
        with charts_output:
            create_system_charts(analysis)
            
    except Exception as e:
        with system_params_output:
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Calculation Error:</strong><br>
                An error occurred during calculation: {str(e)}
            </div>
            """))

# print("Button handler function defined successfully!")


In [327]:
# Connect the button to the calculation function
calculate_button.on_click(on_calculate_click)

# Create the main interface layout
interface_box = widgets.VBox([
    widgets.HBox([
        widgets.VBox([power_widget, t1_widget], layout=widgets.Layout(margin='0 20px 0 0')),
        widgets.VBox([temp_diff_widget, approach_widget])
    ]),
    widgets.HBox([calculate_button]),
    system_params_output,
    cost_analysis_output,
    charts_output
], layout=widgets.Layout(border='2px solid #ddd', padding='20px', margin='10px 0', border_radius='10px'))

# Display the new interface
display(interface_box)

# print("Interface displayed successfully!")


In [ ]:
# Debug Calculations and Lookup Tracing
# This cell shows step-by-step what calculations are being performed

def debug_system_analysis(power, t1, temp_diff, approach):
    """
    Debug version of get_complete_system_analysis that shows every step.
    """
    print("="*80)
    print(f"🔍 DEBUG: HEAT REUSE SYSTEM ANALYSIS")
    print("="*80)
    print(f"Input Parameters:")
    print(f"  Power/Capacity: {power} MW")
    print(f"  T1 Temperature: {t1}°C")
    print(f"  Temperature Difference: {temp_diff}°C")
    print(f"  Approach: {approach}")
    print(f"  Calculated T2: {t1 + temp_diff}°C")
    print()
    
    # Step 1: Validate inputs
    print("STEP 1: Validating Inputs")
    print("-" * 40)
    errors = validate_user_inputs(power, t1, temp_diff, approach)
    if errors:
        print(f"❌ Validation Errors: {errors}")
        return None
    else:
        print("✅ All inputs are valid")
    print()
    
    # Step 2: Check CSV data availability
    print("STEP 2: Checking CSV Data Availability")
    print("-" * 40)
    global csv_data
    print(f"Available CSV files: {list(csv_data.keys())}")
    
    required_csvs = ['ALLHX', 'PIPSZ', 'ROOM', 'PIPCOST', 'CVALV', 'IVALV']
    for csv_name in required_csvs:
        if csv_name in csv_data:
            rows, cols = csv_data[csv_name].shape
            print(f"✅ {csv_name}: {rows} rows, {cols} columns")
        else:
            print(f"❌ {csv_name}: NOT FOUND")
    print()
    
    # Step 3: ALLHX Lookup (most critical)
    print("STEP 3: ALLHX Lookup")
    print("-" * 40)
    t2 = t1 + temp_diff
    
    if 'ALLHX' not in csv_data:
        print("❌ ALLHX.csv not loaded - cannot proceed")
        return None
    
    df = csv_data['ALLHX']
    print(f"ALLHX DataFrame info:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")
    print()
    
    # Show the filtering process
    print(f"Filtering ALLHX for:")
    print(f"  wha (power) = {power}")
    print(f"  T1 = {t1}")
    print(f"  itdt (temp_diff) = {temp_diff}")
    print(f"  T2 = {t2}")
    print(f"  TCSapp (approach) = {approach}")
    print()
    
    # Check each filter condition individually
    power_match = df['wha'] == power
    t1_match = df['T1'] == t1
    itdt_match = df['itdt'] == temp_diff
    t2_match = df['T2'] == t2
    approach_match = df['TCSapp'] == approach
    
    print(f"Filter results:")
    print(f"  Power matches: {power_match.sum()} rows")
    print(f"  T1 matches: {t1_match.sum()} rows")
    print(f"  itdt matches: {itdt_match.sum()} rows")
    print(f"  T2 matches: {t2_match.sum()} rows")
    print(f"  TCSapp matches: {approach_match.sum()} rows")
    
    # Show some sample data to understand the structure
    print(f"\nSample ALLHX data (first 5 rows):")
    print(df[['wha', 'T1', 'itdt', 'T2', 'TCSapp', 'F1', 'F2', 'T3', 'T4']].head())
    print()
    
    # Apply all filters
    filtered_df = df[power_match & t1_match & itdt_match & t2_match & approach_match]
    print(f"Combined filter result: {len(filtered_df)} rows")
    
    if filtered_df.empty:
        print("❌ No matching data found in ALLHX")
        print("\nLet's check what values ARE available:")
        print(f"Available wha values: {sorted(df['wha'].dropna().unique())}")
        print(f"Available T1 values: {sorted(df['T1'].dropna().unique())}")
        print(f"Available itdt values: {sorted(df['itdt'].dropna().unique())}")
        print(f"Available TCSapp values: {sorted(df['TCSapp'].dropna().unique())}")
        return None
    
    # Get the first matching row
    row = filtered_df.iloc[0]
    print(f"✅ Found matching row:")
    print(f"  T3: {row['T3']}")
    print(f"  T4: {row['T4']}")
    print(f"  F1: {row['F1']}")
    print(f"  F2: {row['F2']}")
    print()
    
    # Extract system parameters
    system_data = {
        'power': power,
        'T1': t1,
        'T2': t2,
        'T3': row['T3'],
        'T4': row['T4'],
        'F1': row['F1'],
        'F2': row['F2'],
        'approach_tcs': row['TCSapp'],
        'approach_fws': row['FWSapp'],
        'hx_unit': row['Unit'],
        'hx_cost': row['costHX'],
        'hx_area': row['areaHX'],
        'hx_weight': row['Hxweight'],
        'co2_footprint': row['CO2_Footprint']
    }
    
    # Step 4: System Sizing
    print("STEP 4: System Sizing Calculations")
    print("-" * 40)
    
    # Pipe size lookups
    print(f"Looking up pipe sizes:")
    print(f"  F1 = {system_data['F1']} l/m")
    print(f"  F2 = {system_data['F2']} l/m")
    
    if 'PIPSZ' in csv_data:
        pipsz_df = csv_data['PIPSZ']
        print(f"PIPSZ DataFrame shape: {pipsz_df.shape}")
        print(f"PIPSZ columns: {list(pipsz_df.columns)}")
        print(f"PIPSZ sample data:")
        print(pipsz_df.head())
        
        pipe_size_f1 = get_lookup_value("PIPSZ", system_data['F1'], 0, 1)
        pipe_size_f2 = get_lookup_value("PIPSZ", system_data['F2'], 0, 1)
        
        print(f"  Pipe size for F1 ({system_data['F1']}): {pipe_size_f1}")
        print(f"  Pipe size for F2 ({system_data['F2']}): {pipe_size_f2}")
    else:
        print("❌ PIPSZ not available")
        pipe_size_f1 = pipe_size_f2 = None
    
    # Room size lookup
    print(f"\nLooking up room size for {power} MW:")
    if 'ROOM' in csv_data:
        room_df = csv_data['ROOM']
        print(f"ROOM DataFrame shape: {room_df.shape}")
        print(f"ROOM columns: {list(room_df.columns)}")
        print(f"ROOM data:")
        print(room_df)
        
        room_size = get_lookup_value("ROOM", power)
        print(f"  Room size for {power} MW: {room_size}")
    else:
        print("❌ ROOM not available")
        room_size = None
    
    sizing_data = {
        'pipe_size_f1': pipe_size_f1,
        'pipe_size_f2': pipe_size_f2,
        'room_size': room_size,
        'primary_pipe_size': max(pipe_size_f1 or 0, pipe_size_f2 or 0)
    }
    print(f"Primary pipe size: {sizing_data['primary_pipe_size']}")
    print()
    
    # Step 5: Cost Calculations
    print("STEP 5: Cost Calculations")
    print("-" * 40)
    
    primary_pipe_size = sizing_data['primary_pipe_size']
    
    # Pipe costs
    if 'PIPCOST' in csv_data:
        pipcost_df = csv_data['PIPCOST']
        print(f"PIPCOST DataFrame shape: {pipcost_df.shape}")
        print(f"PIPCOST columns: {list(pipcost_df.columns)}")
        print(f"PIPCOST sample data:")
        print(pipcost_df.head())
        
        pipe_cost_per_meter = get_lookup_value("PIPCOST", primary_pipe_size, 0, 1)
        print(f"  Pipe cost per meter for size {primary_pipe_size}: €{pipe_cost_per_meter}")
    else:
        print("❌ PIPCOST not available")
        pipe_cost_per_meter = 0
    
    # Calculate total pipe length and cost
    total_pipe_length = (room_size * 3) if room_size else 0
    total_pipe_cost = pipe_cost_per_meter * total_pipe_length if pipe_cost_per_meter else 0
    print(f"  Total pipe length (room_size * 3): {total_pipe_length} m")
    print(f"  Total pipe cost: €{total_pipe_cost}")
    
    # Valve costs
    print(f"\nValve cost lookups for pipe size {primary_pipe_size}:")
    
    if 'CVALV' in csv_data:
        cvalv_df = csv_data['CVALV']
        print(f"CVALV DataFrame shape: {cvalv_df.shape}")
        print(f"CVALV columns: {list(cvalv_df.columns)}")
        print(f"CVALV sample data:")
        print(cvalv_df.head())
        
        control_valve_cost = get_exact_match("CVALV", str(int(primary_pipe_size)), 0, 1) or 0
        print(f"  Control valve cost: €{control_valve_cost}")
    else:
        print("❌ CVALV not available")
        control_valve_cost = 0
    
    if 'IVALV' in csv_data:
        ivalv_df = csv_data['IVALV']
        print(f"IVALV DataFrame shape: {ivalv_df.shape}")
        print(f"IVALV columns: {list(ivalv_df.columns)}")
        print(f"IVALV sample data:")
        print(ivalv_df.head())
        
        isolation_valve_cost = get_exact_match("IVALV", str(int(primary_pipe_size)), 0, 1) or 0
        print(f"  Isolation valve cost: €{isolation_valve_cost}")
    else:
        print("❌ IVALV not available")
        isolation_valve_cost = 0
    
    total_valve_cost = (control_valve_cost + isolation_valve_cost) * 4
    print(f"  Total valve cost (4 of each): €{total_valve_cost}")
    
    # Other costs
    hx_cost = system_data['hx_cost']
    pump_cost = system_data['power'] * 5000  # Estimated
    installation_cost = 10000  # Placeholder
    total_cost = total_pipe_cost + total_valve_cost + hx_cost + pump_cost + installation_cost
    
    print(f"\nCost Summary:")
    print(f"  Heat exchanger cost: €{hx_cost}")
    print(f"  Pump cost (estimated): €{pump_cost}")
    print(f"  Installation cost (fixed): €{installation_cost}")
    print(f"  TOTAL SYSTEM COST: €{total_cost}")
    
    cost_data = {
        'pipe_cost_per_meter': pipe_cost_per_meter,
        'total_pipe_length': total_pipe_length,
        'total_pipe_cost': total_pipe_cost,
        'control_valve_cost': control_valve_cost,
        'isolation_valve_cost': isolation_valve_cost,
        'total_valve_cost': total_valve_cost,
        'hx_cost': hx_cost,
        'pump_cost': pump_cost,
        'installation_cost': installation_cost,
        'total_cost': total_cost
    }
    
    # Final result
    complete_analysis = {
        'system': system_data,
        'sizing': sizing_data,
        'costs': cost_data,
        'summary': {
            'power_mw': system_data['power'],
            't1_celsius': system_data['T1'],
            't2_celsius': system_data['T2'],
            't3_celsius': system_data['T3'],
            't4_celsius': system_data['T4'],
            'f1_flow': system_data['F1'],
            'f2_flow': system_data['F2'],
            'pipe_size': sizing_data['primary_pipe_size'],
            'room_size': sizing_data['room_size'],
            'total_cost_eur': round(cost_data['total_cost'])
        }
    }
    
    print("\n" + "="*80)
    print("✅ DEBUG ANALYSIS COMPLETE")
    print("="*80)
    
    return complete_analysis

# Test function - run this to debug with current widget values
def test_current_selections():
    """Test the debug function with current widget selections"""
    power = power_widget.value
    t1 = t1_widget.value
    temp_diff = temp_diff_widget.value
    approach = approach_widget.value
    
    print(f"Testing with current widget selections:")
    print(f"Power: {power}, T1: {t1}, Temp Diff: {temp_diff}, Approach: {approach}")
    print()
    
    result = debug_system_analysis(power, t1, temp_diff, approach)
    return result

# Create an output widget for debug results
debug_output = widgets.Output()

# Add a debug button to the interface
debug_button = widgets.Button(
    description='🔍 Debug Current Selection',
    button_style='info',
    tooltip='Run detailed debug analysis on current selections',
    layout=widgets.Layout(width='220px', height='35px')
)

def on_debug_click(b):
    with debug_output:
        clear_output()
        test_current_selections()

debug_button.on_click(on_debug_click)

# print("Debug functions created!")
# print("Run test_current_selections() to debug with current widget values")
# print("Or click the debug button below to see debug output")

# Display the debug interface
debug_interface = widgets.VBox([
    debug_button,
    debug_output
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0'))

display(debug_interface)

In [ ]:
# COMPREHENSIVE TEST SUITE
# Run this cell to validate all conversions and catch issues

import pandas as pd

def test_number_conversions():
    """Test the universal parser with various edge cases"""
    print("🧪 TESTING NUMBER CONVERSIONS")
    print("="*60)
    
    # Test cases with expected results
    test_cases = [
        # (input, expected_output, description)
        ("1,493", 1493.0, "Your CSV data - comma thousands"),
        ("1,440", 1440.0, "Your CSV data - comma thousands"),
        ("11,024", 11024.0, "Error case from before"),
        ("1.493", 1.493, "European decimal"),
        ("1.234,56", 1234.56, "European thousands+decimal"),
        ("1,234.56", 1234.56, "American thousands+decimal"),
        ("$25,000.50", 25000.50, "Currency format"),
        ("15%", 0.15, "Percentage"),
        ("", 0.0, "Empty string"),
        ("N/A", 0.0, "Text value"),
        ("12345", 12345.0, "Simple integer"),
        ("123.45", 123.45, "Simple decimal"),
    ]
    
    errors = []
    
    for input_val, expected, description in test_cases:
        try:
            result = universal_float_convert(input_val)
            if abs(result - expected) > 0.01:  # Allow small floating point differences
                errors.append(f"❌ {description}: '{input_val}' -> {result} (expected {expected})")
            else:
                print(f"✅ {description}: '{input_val}' -> {result}")
        except Exception as e:
            errors.append(f"💥 {description}: '{input_val}' -> ERROR: {e}")
    
    if errors:
        print("\n🚨 CONVERSION ERRORS FOUND:")
        for error in errors:
            print(error)
    else:
        print("\n🎉 ALL NUMBER CONVERSIONS PASSED!")
    
    return len(errors) == 0

def test_csv_data_integrity():
    """Test that CSV data is loaded and accessible"""
    print("\n🗃️ TESTING CSV DATA INTEGRITY")
    print("="*60)
    
    required_csvs = ['ALLHX', 'PIPSZ', 'ROOM', 'PIPCOST', 'CVALV', 'IVALV']
    issues = []
    
    global csv_data
    
    for csv_name in required_csvs:
        if csv_name not in csv_data:
            issues.append(f"❌ {csv_name}: NOT FOUND")
        else:
            df = csv_data[csv_name]
            rows, cols = df.shape
            
            # Check for minimum data
            if rows < 2:
                issues.append(f"⚠️ {csv_name}: Only {rows} rows (may be header only)")
            elif rows < 5:
                issues.append(f"⚠️ {csv_name}: Only {rows} rows (very small dataset)")
            else:
                print(f"✅ {csv_name}: {rows} rows, {cols} columns")
            
            # Check for data types
            if csv_name == 'ALLHX':
                # Check critical columns exist
                critical_cols = ['wha', 'T1', 'itdt', 'T2', 'TCSapp', 'F1', 'F2', 'T3', 'T4']
                missing_cols = [col for col in critical_cols if col not in df.columns]
                if missing_cols:
                    issues.append(f"❌ {csv_name}: Missing columns: {missing_cols}")
    
    if issues:
        print("\n🚨 CSV DATA ISSUES:")
        for issue in issues:
            print(issue)
        return False
    else:
        print("\n🎉 ALL CSV DATA CHECKS PASSED!")
        return True

def test_lookup_scenarios():
    """Test various lookup scenarios to catch edge cases"""
    print("\n🔍 TESTING LOOKUP SCENARIOS")
    print("="*60)
    
    # Test cases: (power, t1, temp_diff, approach, should_find_data)
    test_scenarios = [
        (1, 20, 10, 2, True, "Basic valid case"),
        (1, 20, 12, 2, True, "Different temp diff"),
        (2, 30, 10, 3, True, "Different power/temp"),
        (5, 45, 14, 5, True, "Maximum values"),
        (1, 25, 10, 2, False, "Invalid T1 (not in dropdown)"),
        (6, 20, 10, 2, False, "Invalid power (not in dropdown)"),
        (1, 20, 15, 2, False, "Invalid temp diff (not in dropdown)"),
        (1, 20, 10, 4, False, "Invalid approach (not in dropdown)"),
    ]
    
    passed = 0
    failed = 0
    
    for power, t1, temp_diff, approach, should_find, description in test_scenarios:
        try:
            result = get_complete_system_analysis(power, t1, temp_diff, approach)
            
            if should_find and result:
                print(f"✅ {description}: Found data as expected")
                passed += 1
            elif not should_find and not result:
                print(f"✅ {description}: No data found as expected")
                passed += 1
            elif should_find and not result:
                print(f"❌ {description}: Expected data but found none")
                failed += 1
            else:
                print(f"⚠️ {description}: Found data when shouldn't have")
                failed += 1
                
        except Exception as e:
            print(f"💥 {description}: ERROR - {e}")
            failed += 1
    
    print(f"\n📊 LOOKUP TESTS: {passed} passed, {failed} failed")
    return failed == 0

def test_data_ranges():
    """Test that converted values are in reasonable ranges"""
    print("\n📏 TESTING DATA RANGES")
    print("="*60)
    
    # Test with a known good case
    result = get_complete_system_analysis(1, 20, 10, 2)
    
    if not result:
        print("❌ Cannot test ranges - no data returned")
        return False
    
    issues = []
    system = result['system']
    sizing = result['sizing'] 
    costs = result['costs']
    
    # Temperature range checks
    if not (0 <= system['T3'] <= 100):
        issues.append(f"T3 out of range: {system['T3']}°C")
    if not (0 <= system['T4'] <= 100):
        issues.append(f"T4 out of range: {system['T4']}°C")
    
    # Flow rate range checks (should be reasonable for HVAC)
    if not (10 <= system['F1'] <= 50000):
        issues.append(f"F1 out of reasonable range: {system['F1']} l/m")
    if not (10 <= system['F2'] <= 50000):
        issues.append(f"F2 out of reasonable range: {system['F2']} l/m")
    
    # Cost range checks
    if not (0 <= costs['hx_cost'] <= 1000000):
        issues.append(f"HX cost unreasonable: €{costs['hx_cost']}")
    if not (0 <= costs['total_cost'] <= 5000000):
        issues.append(f"Total cost unreasonable: €{costs['total_cost']}")
    
    # Pipe size checks
    if not (50 <= sizing['primary_pipe_size'] <= 1000):
        issues.append(f"Pipe size unreasonable: {sizing['primary_pipe_size']}")
    
    # Room size checks
    if not (5 <= sizing['room_size'] <= 100):
        issues.append(f"Room size unreasonable: {sizing['room_size']} m")
    
    if issues:
        print("🚨 DATA RANGE ISSUES:")
        for issue in issues:
            print(f"❌ {issue}")
        return False
    else:
        print("✅ All values are in reasonable ranges")
        print(f"   T3: {system['T3']}°C, T4: {system['T4']}°C")
        print(f"   F1: {system['F1']:,.0f} l/m, F2: {system['F2']:,.0f} l/m")
        print(f"   Pipe size: {sizing['primary_pipe_size']}, Room: {sizing['room_size']} m")
        print(f"   HX cost: €{costs['hx_cost']:,.0f}, Total: €{costs['total_cost']:,.0f}")
        return True

def test_calculation_consistency():
    """Test that calculations are consistent across different scenarios"""
    print("\n🔄 TESTING CALCULATION CONSISTENCY")
    print("="*60)
    
    # Test multiple scenarios and check for consistency
    scenarios = [
        (1, 20, 10, 2),
        (2, 30, 12, 3),
        (3, 45, 14, 5),
    ]
    
    results = []
    issues = []
    
    for power, t1, temp_diff, approach in scenarios:
        result = get_complete_system_analysis(power, t1, temp_diff, approach)
        if result:
            results.append((power, t1, temp_diff, approach, result))
            print(f"✅ Scenario {power}MW, {t1}°C, +{temp_diff}°C, approach {approach}: Success")
        else:
            issues.append(f"❌ No data for {power}MW, {t1}°C, +{temp_diff}°C, approach {approach}")
    
    # Check consistency rules
    if len(results) >= 2:
        # Higher power should generally mean higher costs
        for i in range(len(results)-1):
            curr_power = results[i][0]
            next_power = results[i+1][0]
            curr_cost = results[i][4]['costs']['total_cost']
            next_cost = results[i+1][4]['costs']['total_cost']
            
            if next_power > curr_power and next_cost < curr_cost:
                issues.append(f"⚠️ Higher power ({next_power}MW) has lower cost (€{next_cost:,.0f}) than lower power ({curr_power}MW: €{curr_cost:,.0f})")
    
    if issues:
        print("\n🚨 CONSISTENCY ISSUES:")
        for issue in issues:
            print(issue)
        return False
    else:
        print("\n🎉 CALCULATIONS ARE CONSISTENT!")
        return True

def test_widget_integration():
    """Test that the system works with current widget values"""
    print("\n🎛️ TESTING WIDGET INTEGRATION")
    print("=" * 40)
    
    # Get current widget values
    power = power_widget.value
    t1 = t1_widget.value
    temp_diff = temp_diff_widget.value
    approach = approach_widget.value
    
    print(f"Current widget values: {power}MW, {t1}°C, +{temp_diff}°C, approach {approach}")
    
    # Test with these values
    result = get_complete_system_analysis(power, t1, temp_diff, approach)
    
    if result:
        print("✅ Widget integration successful!")
        print(f"Total cost: €{result['costs']['total_cost']:,.0f}")
        return True
    else:
        print("❌ Widget integration failed")
        return False

    # Run the complete fix tests
    test_complete_system()
    test_widget_integration()


def run_full_test_suite():
    """Run all tests and provide summary"""
    print("🧪 RUNNING COMPREHENSIVE TEST SUITE")
    print("="*80)
    print("This will test number parsing, CSV data, lookups, ranges, and consistency")
    print("="*80)
    
    tests = [
        ("Number Conversions", test_number_conversions),
        ("CSV Data Integrity", test_csv_data_integrity),
        ("Lookup Scenarios", test_lookup_scenarios),
        ("Data Ranges", test_data_ranges),
        ("Calculation Consistency", test_calculation_consistency),
        ("Widget Integration", test_widget_integration),
    ]
    
    passed_tests = 0
    total_tests = len(tests)
    
    for test_name, test_func in tests:
        print(f"\n{'='*20} {test_name} {'='*20}")
        try:
            if test_func():
                passed_tests += 1
                print(f"🎉 {test_name}: PASSED")
            else:
                print(f"❌ {test_name}: FAILED")
        except Exception as e:
            print(f"💥 {test_name}: ERROR - {e}")
    
    print("\n" + "="*80)
    print("📊 TEST SUITE SUMMARY")
    print("="*80)
    print(f"Total Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")
    
    if passed_tests == total_tests:
        print("\n🎉 ALL TESTS PASSED! System is ready for production use!")
    else:
        print(f"\n⚠️ {total_tests - passed_tests} tests failed. Review issues above.")
    
    return passed_tests == total_tests

# Create test buttons
run_full_test_button = widgets.Button(
    description='🧪 Run Full Test Suite',
    button_style='info',
    tooltip='Run comprehensive validation tests',
    layout=widgets.Layout(width='200px', height='40px')
)

quick_test_button = widgets.Button(
    description='⚡ Quick Test',
    button_style='warning', 
    tooltip='Run basic validation tests',
    layout=widgets.Layout(width='150px', height='40px')
)

test_output = widgets.Output()

def on_full_test_click(b):
    with test_output:
        clear_output()
        run_full_test_suite()

def on_quick_test_click(b):
    with test_output:
        clear_output()
        print("⚡ RUNNING QUICK TESTS")
        print("="*40)
        
        # Run just the essential tests
        conv_ok = test_number_conversions()
        csv_ok = test_csv_data_integrity()
        lookup_ok = test_lookup_scenarios()
        
        print("\n📊 QUICK TEST SUMMARY")
        print("="*40)
        tests_passed = sum([conv_ok, csv_ok, lookup_ok])
        print(f"Passed: {tests_passed}/3")
        
        if tests_passed == 3:
            print("🎉 Quick tests passed! System looks good.")
        else:
            print("⚠️ Some quick tests failed. Run full test suite for details.")

run_full_test_button.on_click(on_full_test_click)
quick_test_button.on_click(on_quick_test_click)

print("🧪 COMPREHENSIVE TEST SUITE READY!")
print("="*50)
print("Two testing options available:")
print("  • Quick Test: Essential validation (faster)")
print("  • Full Test Suite: Complete validation (thorough)")
print("="*50)

# Display test interface
test_interface = widgets.VBox([
    widgets.HTML("<h3>🧪 System Validation Tests</h3>"),
    widgets.HBox([quick_test_button, run_full_test_button]),
    test_output
], layout=widgets.Layout(border='2px solid #9C27B0', padding='15px', margin='10px 0'))



display(test_interface)

🧪 COMPREHENSIVE TEST SUITE READY!
Two testing options available:
  • Quick Test: Essential validation (faster)
  • Full Test Suite: Complete validation (thorough)
